<a href="https://colab.research.google.com/github/rurusasu/RecommendSystem/blob/main/%E3%83%A6%E3%83%BC%E3%82%B6%E3%83%99%E3%83%BC%E3%82%B9%E3%81%AE%E5%8D%94%E8%AA%BF%E3%83%95%E3%82%A3%E3%83%AB%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ユーザベースの協調フィルタリング を試験的に作成する

参考
* [レコメンデーション入門2　協調フィルタリング](https://qiita.com/ngayope330/items/fa1865d2952714cce86d)

In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dir = "/content/drive/MyDrive/Google AI Studio"

In [ ]:
!pip install  --upgrade -q tensorflow_recommenders tensorflow-datasets apache-beam xlearn fastFM

## ライブラリ読み込み

In [ ]:
import os
import tempfile

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import matplotlib.pyplot as plt
#import xlearn as xl
from fastFM import als
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder, StandardScaler
from IPython.display import clear_output
from scipy.sparse import csr_matrix

plt.style.use("seaborn-whitegrid")

<ipython-input-3-871df6646d70>:21: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


## データの読み込みと前処理

### データの読み込み

In [ ]:
# エクセルファイルからデータを読み込む
data = pd.read_excel(f"{base_dir}/sample_merged_full.xlsx")
data.head(2)

,user_id,target_id,rating,rating_conv,user_name_target,nickname_target,gender_target,location_target,age_range_target,height_range_target,...,body_type_user,personality_user,appearance_user,job_user,blood_type_user,car_user,interests_user,salary_user,plan_user,account_creation_timestamp_user
0,1,8627.0,0.0,1,原田遥,アオイ,女性,埼玉県伊奈町,45-49,150-154,...,スリム,元気,セクシー系,会社員,O型,有り,技術・プログラミング,8160000,option2,2024-01-14 00:11:34
1,1,18213.0,0.0,1,井上萌,ユイ,女性,福島県玉川村,30-34,150-154,...,スリム,元気,セクシー系,会社員,O型,有り,技術・プログラミング,8160000,option2,2024-01-14 00:11:34


In [ ]:
# NaNを含むレコードを削除
cleaned_data_selected = data.dropna()

# 必要な列を選択
user_features = cleaned_data_selected[['user_id', 'age_range_user', 'height_range_user', 'body_type_user',
                    'personality_user', 'appearance_user', 'job_user', 'blood_type_user',
                    'car_user', 'interests_user', 'salary_user', 'plan_user']]
target_features = cleaned_data_selected[['target_id', 'age_range_target', 'height_range_target', 'body_type_target',
                      'personality_target', 'appearance_target', 'job_target', 'blood_type_target',
                      'car_target', 'interests_target', 'salary_target', 'plan_target']]
ratings = cleaned_data_selected[['user_id', 'target_id', 'rating_conv']]

### レコード数の確認

In [ ]:
# user_features全体の欠損値の個数を確認
total_na = user_features.isna().sum().sum()
print(f"Total missing values in DataFrame: {total_na}")
# レコード数を確認
record_count = len(user_features)
print(f"Number of records: {record_count}")

Total missing values in DataFrame: 0
Number of records: 998


In [ ]:
# user_features全体の欠損値の個数を確認
total_na = target_features.isna().sum().sum()
print(f"Total missing values in DataFrame: {total_na}")
# レコード数を確認
record_count = len(target_features)
print(f"Number of records: {record_count}")

Total missing values in DataFrame: 0
Number of records: 998


In [ ]:
# ratings全体の欠損値の個数を確認
total_na = ratings.isna().sum().sum()
print(f"Total missing values in DataFrame: {total_na}")
# レコード数を確認
record_count = len(ratings)
print(f"Number of records: {record_count}")

Total missing values in DataFrame: 0
Number of records: 998


### カテゴリカルデータを数値データに変換

In [ ]:
label_encoders = {}

# user_features のカテゴリカルデータを数値データに変換
user_features_copy = user_features.copy()
for column in user_features_copy.columns:
    if user_features_copy[column].dtype == 'object':
        le = LabelEncoder()
        user_features_copy[column] = le.fit_transform(user_features_copy[column].astype(str))
        label_encoders[column] = le

# target_features のカテゴリカルデータを数値データに変換
target_features_copy = target_features.copy()
for column in target_features_copy.columns:
    if target_features_copy[column].dtype == 'object':
        le = LabelEncoder()
        target_features_copy[column] = le.fit_transform(target_features_copy[column].astype(str))
        label_encoders[column] = le

### 特徴量の標準化

In [ ]:
# 特徴量を標準化
scaler = StandardScaler()
user_features_scaled = scaler.fit_transform(user_features_copy.drop(columns=['user_id']))
target_features_scaled = scaler.fit_transform(target_features_copy.drop(columns=['target_id']))

### ユーザー特徴量とターゲット特徴量を結合

In [ ]:
# ユーザー特徴量とターゲット特徴量を結合
user_features_scaled_df = pd.DataFrame(user_features_scaled, columns=user_features.columns[1:])
target_features_scaled_df = pd.DataFrame(target_features_scaled, columns=target_features.columns[1:])
user_features_scaled_df['user_id'] = user_features['user_id']
target_features_scaled_df['target_id'] = target_features['target_id']

# 評価データに対応する特徴量をマージ
merged_data = ratings.merge(user_features_scaled_df, on='user_id').merge(target_features_scaled_df, on='target_id')
print(len(merged_data))
merged_data.head(2)

13610


,user_id,target_id,rating_conv,age_range_user,height_range_user,body_type_user,personality_user,appearance_user,job_user,blood_type_user,...,height_range_target,body_type_target,personality_target,appearance_target,job_target,blood_type_target,car_target,interests_target,salary_target,plan_target
0,1,8627.0,1,-1.29508,1.656003,-0.627816,-0.04794,0.127649,-0.406456,1.502784,...,-1.404534,0.43349,1.43928,-1.680975,-0.352382,-0.63815,0.663412,-1.518091,1.639934,1.035693
1,1,8627.0,1,-1.29508,1.656003,-0.627816,-0.04794,0.127649,-0.406456,1.502784,...,-1.404534,0.43349,1.43928,-1.680975,-0.352382,-0.63815,0.663412,-1.518091,1.639934,1.035693


# 標準的な協調フィルタリングアルゴリズム

## 類似度を計算するための関数

In [ ]:
def similarity(data1_dict: dict, data2_dict: dict) -> float:
    """
    2つの辞書間のコサイン類似度を計算します。

    引数:
        data1_dict (dict): 特徴がキー、特徴の重みが値となる最初の辞書。
        data2_dict (dict): 特徴がキー、特徴の重みが値となる2番目の辞書。

    戻り値:
        float: 2つの辞書間のコサイン類似度。

    例外:
        TypeError: 入力のどちらかが辞書でない場合に発生します。
        ValueError: 辞書間に共通のキーがない場合、またはベクトルの大きさがゼロである場合に発生します。
    """

    # 入力が辞書型であるかを確認します。
    if not isinstance(data1_dict, dict) or not isinstance(data2_dict, dict):
        raise TypeError("両方の入力は辞書でなければなりません。")

    # 共通のキーを抽出します。
    same_key = set(data1_dict.keys()) & set(data2_dict.keys())

    # 共通のキーが存在しない場合はエラーを発生させます。
    if not same_key:
        raise ValueError("辞書には共通のキーがありません。")

    # ベクトル同士の内積を求めます。
    ab = 0
    for key in same_key:
        ab += data1_dict[key] * data2_dict[key]

    # 各ベクトルの大きさを求めます。
    a = math.sqrt(sum([data1_dict[key] ** 2 for key in same_key]))
    b = math.sqrt(sum([data2_dict[key] ** 2 for key in same_key]))

    # ベクトルの大きさがゼロである場合はエラーを発生させます。
    if a == 0 or b == 0:
        raise ValueError("ベクトルの大きさがゼロであるため、類似度を計算できません。")

    # コサイン類似度を計算して返します。
    return ab / (a * b)

In [ ]:
def sim_pearson(data1_dict: dict, data2_dict: dict) -> float:
    """
    2つの辞書間のピアソンの相関係数を計算します。

    引数:
        data1_dict (dict): 特徴がキー、特徴の値が値となる最初の辞書。
        data2_dict (dict): 特徴がキー、特徴の値が値となる2番目の辞書。

    戻り値:
        float: 2つの辞書間のピアソンの相関係数。

    例外:
        TypeError: 入力のどちらかが辞書でない場合に発生します。
        ValueError: 辞書間に共通のキーがない場合に発生します。
    """

    # 入力が辞書型であるかを確認します。
    if not isinstance(data1_dict, dict) or not isinstance(data2_dict, dict):
        raise TypeError("両方の入力は辞書でなければなりません。")

    # 共通のキーを抽出します。
    same_key = set(data1_dict.keys()) & set(data2_dict.keys())

    # 共通のキーが存在しない場合はエラーを発生させます。
    if not same_key:
        raise ValueError("辞書には共通のキーがありません。")

    # 共通のキーを持つ要素のリストを作成します。
    values1 = np.array([data1_dict[key] for key in same_key])
    values2 = np.array([data2_dict[key] for key in same_key])

    # それぞれの平均と標準偏差を求めます。
    mean_1 = np.mean(values1)
    std_1 = np.std(values1)
    mean_2 = np.mean(values2)
    std_2 = np.std(values2)

    # 標準偏差がゼロである場合は相関係数を計算できないためエラーを発生させます。
    if std_1 == 0 or std_2 == 0:
        raise ValueError("標準偏差がゼロであるため、相関係数を計算できません。")

    # 共分散を求めます。
    cov = np.sum((values1 - mean_1) * (values2 - mean_2))

    # ピアソンの相関係数を求めます。
    corr = cov / (std_1 * std_2)

    return corr

In [ ]:
def spearman(data1_dict: dict, data2_dict: dict) -> float:
    """
    2つの辞書間のスピアマンの順位相関係数を計算します。

    引数:
        data1_dict (dict): 特徴がキー、特徴の順位が値となる最初の辞書。
        data2_dict (dict): 特徴がキー、特徴の順位が値となる2番目の辞書。

    戻り値:
        float: 2つの辞書間のスピアマンの順位相関係数。

    例外:
        TypeError: 入力のどちらかが辞書でない場合に発生します。
        ValueError: 辞書間に共通のキーがない場合、または共通のキーの数が2未満の場合に発生します。
        ValueError: ベクトルの大きさがゼロの場合に発生します。
    """

    # 入力が辞書型であるかを確認します。
    if not isinstance(data1_dict, dict) or not isinstance(data2_dict, dict):
        raise TypeError("両方の入力は辞書でなければなりません。")

    # 共通のキーを抽出します。
    same_key = set(data1_dict.keys()) & set(data2_dict.keys())

    # 共通のキーが存在しない場合はエラーを発生させます。
    if not same_key:
        raise ValueError("辞書には共通のキーがありません。")

    # 共通のキーの数が2未満の場合はエラーを発生させます。
    if len(same_key) < 2:
        raise ValueError("共通のキーの数が少なすぎます。")

    # data からそれぞれの順位だけを取り出します。
    X = [data1_dict[key] for key in same_key]
    Y = [data2_dict[key] for key in same_key]

    # X, Y を ndarray に変換します。
    X = np.array(X)
    Y = np.array(Y)

    # 全体の数 N を求めます。
    N = len(X)

    # ベクトルの大きさがゼロである場合はエラーを発生させます。
    if N == 0:
        raise ValueError("ベクトルの大きさがゼロです。")

    # スピアマンの順位相関係数を求めます。
    rank_X = np.argsort(np.argsort(X))
    rank_Y = np.argsort(np.argsort(Y))
    d = rank_X - rank_Y
    d_squared = np.sum(d ** 2)

    return 1 - (6 * d_squared) / (N * (N ** 2 - 1))

## Webで公開されているデータを使って、協調フィルタリングを実施

In [ ]:
import math

dataset={
    'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5,'You, Me and Dupree': 2.5, 'The Night Listener': 3.0},
    'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5,'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5},
    'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,'Superman Returns': 3.5, 'The Night Listener': 4.0},
    'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5,'Superman Returns': 4.0, 'You, Me and Dupree': 2.5},
    'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0,'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0},
    'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0,'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
    'Toby': {'Snakes on a Plane':4.5, 'You, Me and Dupree':1.0, 'Superman Returns':4.0}
}

In [ ]:
def get_recommend(target):
    #映画ごとにtotalに類似度×評価、sim_sumsに類似度の合計を入れていきます。
    totals = {}
    sim_sums = {}

    #tardet以外のユーザーのリストを作ってください。
    list_person = set(dataset.keys())
    list_person.remove(target)

    print(list_person)
    print(type(list_person))

    for person in list_person:
        # targetの見ていない映画のリストを作ってください。
        set_person = set(dataset[person])
        set_target = set(dataset[target])
        set_new_movie = set_person - set_target

        #あるユーザとtargetの類似度を計算してください。
        #sim = similarity(dataset[target],dataset[person])
        sim = sim_pearson(dataset[target],dataset[person])
        #sim = spearman(dataset[target],dataset[person])

        for movie in set_new_movie:
            # 類似度 x評価をtotalへ映画ごとに入れてください。
            totals.setdefault(movie,0)
            totals[movie] += dataset[person][movie]*sim

            #類似度の合計を計算するためsim_sumsに類似度を加算してください。
            sim_sums.setdefault(movie,0)
            sim_sums[movie] += sim

    rankings = {movie : total/sim_sums[movie] for movie,total in totals.items()}
    rankings = sorted(rankings.items(), key = lambda x: -x[1])
    return rankings[0][0]

get_recommend('Toby')

{'Michael Phillips', 'Lisa Rose', 'Mick LaSalle', 'Jack Matthews', 'Gene Seymour', 'Claudia Puig'}
<class 'set'>


'The Night Listener'

## 自作データをつかって協調フィルタリングを実施

### 必要なデータの抽出

In [ ]:
df = merged_data[['user_id', 'target_id', 'rating_conv']]
df = df.dropna()
df.head(2)

,user_id,target_id,rating_conv
0,1,8627.0,1
1,1,8627.0,1


In [ ]:
# ピボットテーブルを作成
ratings = df.pivot_table(index='user_id', columns='target_id', values='rating_conv')
# 欠損値を0で埋める
ratings = ratings.fillna(0)
ratings.head(2)

target_id,97.0,128.0,219.0,241.0,282.0,341.0,444.0,532.0,747.0,755.0,...,98842.0,99108.0,99132.0,99289.0,99388.0,99556.0,99606.0,99616.0,99753.0,99827.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 自作データを使って評価する

In [ ]:
def get_recommend(user_id: str) -> str:
    """
    userに対するtargetの推薦を行います。
    """
    # 類似度と評価を格納する辞書
    totals = {}
    sim_sums = {}

    # user_id以外のユーザーリストを作成
    list_person = set(ratings.index) - {user_id}

    for person in list_person:
        # userが評価していないpersonのリストを作成
        user_ratings = ratings.loc[user_id]
        person_ratings = ratings.loc[person]

        # 類似度を計算
        sim = sim_pearson(user_ratings, person_ratings)

        if sim <= 0:  # 類似度が0以下の場合はスキップ
            continue

        for target_id in ratings.columns:
            if user_ratings[target_id] == 0 and person_ratings[target_id] != 0:
                totals.setdefault(target_id, 0)
                totals[target_id] += person_ratings[target_id] * sim
                sim_sums.setdefault(target_id, 0)
                sim_sums[target_id] += sim

    rankings = {target_id: total / sim_sums[target_id] for target_id, total in totals.items() if sim_sums[target_id] != 0}
    rankings = sorted(rankings.items(), key=lambda x: -x[1])

    return rankings if rankings else None

In [ ]:
# テスト例
try:
    recommended_target = get_recommend('3')
    print(f"Recommended target: {recommended_target}")
except Exception as e:
    print(e)

'3'


# Matrix Factorization を使う

## 必要なデータの抽出

In [ ]:
df = merged_data[['user_id', 'target_id', 'rating_conv']]
df = df.dropna()
df.head(2)

,user_id,target_id,rating_conv
0,1,8627.0,1
1,1,8627.0,1


In [ ]:
# ピボットテーブルを作成
ratings = df.pivot_table(index='user_id', columns='target_id', values='rating_conv')
# 欠損値を0で埋める
ratings = ratings.fillna(0)
ratings.head(2)

target_id,97.0,128.0,219.0,241.0,282.0,341.0,444.0,532.0,747.0,755.0,...,98842.0,99108.0,99132.0,99289.0,99388.0,99556.0,99606.0,99616.0,99753.0,99827.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 自作データを使って評価する

In [ ]:
nmf_model = NMF(n_components=2,init='random', random_state=0)
user = nmf_model.fit_transform(ratings)
target = nmf_model.components_

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
from typing import Dict, List

# 評価行列の再構成を計算
reconstructed_ratings = np.dot(user, target)

# 再構成された評価行列をデータフレームに変換
reconstructed_ratings_df = pd.DataFrame(reconstructed_ratings, index=ratings.index, columns=ratings.columns)

def recommend_top_n(reconstructed_ratings_df: pd.DataFrame, n: int = 10) -> Dict[int, List[float]]:
    """
    各ユーザーに対してトップNのターゲットIDを推薦する関数

    Args:
        reconstructed_ratings_df (pd.DataFrame): 再構成された評価行列のデータフレーム
        n (int, optional): 推薦するターゲットIDの数. デフォルトは10.

    Returns:
        Dict[int, List[float]]: 各ユーザーに対するトップNのターゲットIDの辞書
    """
    recommendations = {}
    try:
        for user_id in reconstructed_ratings_df.index:
            user_ratings = reconstructed_ratings_df.loc[user_id]
            top_n_recommendations = user_ratings.nlargest(n).index.tolist()
            recommendations[user_id] = top_n_recommendations
    except Exception as e:
        print(f"Error occurred: {e}")
    return recommendations

# 各ユーザーに対してトップ10の推薦を取得
top_10_recommendations = recommend_top_n(reconstructed_ratings_df, n=10)
print(top_10_recommendations)

{1: [18464.0, 19266.0, 97766.0, 28885.0, 57294.0, 73144.0, 60092.0, 80473.0, 95175.0, 2682.0], 2: [18464.0, 19266.0, 97766.0, 28885.0, 57294.0, 73144.0, 60092.0, 80473.0, 95175.0, 2682.0], 3: [47147.0, 82004.0, 90994.0, 8066.0, 37404.0, 47530.0, 54163.0, 88984.0, 7493.0, 53314.0], 4: [47147.0, 82004.0, 90994.0, 8066.0, 37404.0, 47530.0, 54163.0, 88984.0, 7493.0, 53314.0], 5: [47147.0, 82004.0, 90994.0, 8066.0, 37404.0, 47530.0, 54163.0, 88984.0, 7493.0, 53314.0], 6: [18464.0, 19266.0, 97766.0, 28885.0, 57294.0, 73144.0, 60092.0, 80473.0, 95175.0, 2682.0], 7: [47147.0, 82004.0, 90994.0, 8066.0, 37404.0, 47530.0, 54163.0, 88984.0, 7493.0, 53314.0], 8: [47147.0, 82004.0, 90994.0, 8066.0, 37404.0, 47530.0, 54163.0, 88984.0, 7493.0, 53314.0], 9: [47147.0, 82004.0, 90994.0, 8066.0, 37404.0, 47530.0, 54163.0, 88984.0, 7493.0, 53314.0], 10: [47147.0, 82004.0, 90994.0, 8066.0, 37404.0, 47530.0, 54163.0, 88984.0, 7493.0, 53314.0], 11: [18464.0, 19266.0, 97766.0, 28885.0, 57294.0, 73144.0, 60092.

In [ ]:
import pandas as pd
from typing import List

def recommend_top_n_for_user(reconstructed_ratings_df: pd.DataFrame, user_id: int, n: int = 10) -> List[float]:
    """
    特定のユーザーIDに対してトップNのターゲットIDを推薦する関数

    Args:
        reconstructed_ratings_df (pd.DataFrame): 再構成された評価行列のデータフレーム
        user_id (int): 推薦対象のユーザーID
        n (int, optional): 推薦するターゲットIDの数. デフォルトは10.

    Returns:
        List[float]: 特定のユーザーに対するトップNのターゲットIDのリスト
    """
    try:
        if user_id in reconstructed_ratings_df.index:
            user_ratings = reconstructed_ratings_df.loc[user_id]
            top_n_recommendations = user_ratings.nlargest(n).index.tolist()
            return top_n_recommendations
        else:
            print(f"user_id {user_id} not found in the data")
            return []
    except Exception as e:
        print(f"Error occurred: {e}")
        return []

# 評価行列の再構成を計算
reconstructed_ratings = np.dot(user, target)

# 再構成された評価行列をデータフレームに変換
reconstructed_ratings_df = pd.DataFrame(reconstructed_ratings, index=ratings.index, columns=ratings.columns)

# Get the top 10 recommendations for user_id
user_id_to_recommend = 1
top_10_recommendations_for_user = recommend_top_n_for_user(reconstructed_ratings_df, user_id_to_recommend, n=10)
print(top_10_recommendations_for_user)

[18464.0, 19266.0, 97766.0, 28885.0, 57294.0, 73144.0, 60092.0, 80473.0, 95175.0, 2682.0]


# Factrization Machine を使う

## データの準備

In [ ]:
# データの準備
X = merged_data.drop(columns=['rating_conv']).values
y = merged_data['rating_conv'].values

In [ ]:
# トレーニングとテストの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## FMモデルの訓練

In [ ]:
# FastFM ALSモデルの作成
fm = als.FMRegression(n_iter=2000, rank=2, random_state=42)

# モデルの訓練
fm.fit(csr_matrix(X_train), y_train)

# 予測
y_pred = fm.predict(csr_matrix(X_test))

# モデルの評価
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')

RMSE: 1.4618085127403377


### 推薦の実施

In [ ]:
from typing import List, Any

def recommend_for_user(fm_model: Any, user_id: int, all_targets: List[int], n_recommendations: int = 5) -> List[int]:
    """
    指定されたユーザーに対してターゲットの評価値を予測し、上位のターゲットを推薦する関数。

    Args:
        fm_model (Any): 学習済みのファクタライゼーションマシンモデル。
        user_id (int): 評価を予測するユーザーのID。
        all_targets (List[int]): 評価対象のターゲットのIDリスト。
        n_recommendations (int): 推薦するターゲットの数。デフォルトは5。

    Returns:
        List[int]: 推薦するターゲットのIDリスト。
    """
    try:
        # ユーザーIDに対する全てのターゲットの評価値を予測
        user_data = merged_data[merged_data['user_id'] == user_id].drop(columns=['rating_conv'])

        # ターゲットIDごとに評価値を予測
        predictions = []
        for target_id in all_targets:
            # 特定のターゲットに対する行を抽出
            target_data = user_data[user_data['target_id'] == target_id]
            if not target_data.empty:
                target_features = csr_matrix(target_data.values)
                prediction = fm_model.predict(target_features)[0]
                predictions.append((target_id, prediction))

        # 評価値が高い順にソート
        predictions.sort(key=lambda x: x[1], reverse=True)

        # 上位のターゲットを返す
        top_recommendations = [target for target, _ in predictions[:n_recommendations]]
        return top_recommendations

    except Exception as e:
        print(f"Error in recommending for user {user_id}: {e}")
        return []

In [ ]:
# 全てのターゲットIDを取得
all_targets = merged_data['target_id'].unique()

# 例: user_id に対してトップ10の推薦を生成
user_id = 1
recommendations = recommend_for_user(fm, user_id, all_targets, n_recommendations=10)
print(f"Recommendations for user {user_id}: {recommendations}")

Recommendations for user 1: [23263.0, 44212.0, 40440.0, 40895.0, 49685.0, 48426.0, 18859.0, 85527.0, 36543.0, 18213.0]
